In [1]:
import pandas as pd
import requests
import json
import re
from bs4 import BeautifulSoup
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
matplotlib.rcParams['pdf.fonttype'] = 42

%matplotlib inline

In [2]:
region1 = pd.read_excel("region1_unredacted.xlsx")

In [3]:
region2 = pd.read_excel("region2_unredacted.xlsx")

In [4]:
region3 = pd.read_excel("region3_unredacted.xlsx")

In [5]:
region4 = pd.read_excel("region4_unredacted.xlsx")

In [6]:
region5a = pd.read_excel("region5a_unredacted.xlsx")

In [7]:
region5b = pd.read_excel("region5b_unredacted.xlsx")

In [8]:
region6 = pd.read_excel("region6_unredacted.xlsx")

In [9]:
region7 = pd.read_excel("region7_unredacted.xlsx")

In [10]:
region8 = pd.read_excel("region8_unredacted.xlsx")

In [11]:
region9 = pd.read_excel("region9_unredacted.xlsx")

In [12]:
region10 = pd.read_excel("region10_unredacted.xlsx")

In [13]:
frames = [region1, region2, region3, region4, region5a, region5b, region6, region7, region8, region9, region10]

In [14]:
df = pd.concat(frames)

In [15]:
df.head()

,facility_name,facility_id,address,city,state,zip,inspection_date,deficiency_tag,scope_severity,complaint,standard,eventid,unredacted_inspection_text
0,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,157,D,0,1,XW2G11,13586<BR/>Based on review of the clinical reco...
1,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,166,D,1,1,XW2G11,"><BR/>Based on clinical record review, intervi..."
2,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,241,D,1,1,XW2G11,"><BR/>Based on clinical record review, intervi..."
3,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,253,E,1,1,XW2G11,"><BR/>Based on observations and interview, the..."
4,ST JOSEPH'S CENTER,075001,6448 MAIN STREET,TRUMBULL,CT,6611,11/18/2016,309,D,1,1,XW2G11,><BR/>Based on review of the clinical record a...


In [16]:
urls = pd.read_csv("corp_type.csv")

In [17]:
urls['facility_id'] = urls.URL.str.replace(r'https://www.medicare.gov/nursinghomecompare/profile.html#profTab=0&ID=', '', regex=True)

In [18]:
urls = urls.drop(['URL', 'slug'], axis=1)

In [19]:
df = df.merge(urls, left_on='facility_id', right_on='facility_id')

In [20]:
df['corp_type']=df.corp_type.replace({"For profit - Corporation": "For profit", 
                           "Non profit - Corporation": "Non profit/Government", 
                           "For profit - Limited Liability company": "For profit",
                           "For profit - Partnership": "For profit",
                           "For profit - Individual": "For profit",
                           "Government - County": "Non profit/Government",
                           "Non profit - Other": "Non profit/Government",
                           "Non profit - Church related": "Non profit/Government",
                           "Government - Hospital district": "Non profit/Government",
                           "Government - State": "Non profit/Government",
                           "Government - City/county": "Non profit/Government",
                           "Government - City": "Non profit/Government",
                           "Government - Federal": "Non profit/Government"})

In [57]:
df = df[df.corp_type != 'Poorly formatted :(']


In [61]:
df = df[df.state != 'GU']

## Complaints filed per capita

In [21]:
census_Q1 = pd.read_csv('PBJ_Daily_Nurse_Staffing_CY_2018_Q1.csv')

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
census_Q2 = pd.read_csv('PBJ_Daily_Nurse_Staffing_CY_2018Q2.csv')

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,21,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
census_Q3 = pd.read_csv('PBJ_Daily_Nurse_Staffing_CY_2018Q3.csv')

In [24]:
census_Q4 = pd.read_csv('PBJ_Daily_Nurse_Staffing_CY_2018Q4.csv')

In [25]:
frames = [census_Q1, census_Q2, census_Q3, census_Q4]

In [26]:
staffing = pd.concat(frames)

/Users/elenamlutz/.pyenv/versions/3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [62]:
staffing = staffing[['WorkDate', 'PROVNUM', 'PROVNAME', 'STATE', 'MDScensus', 'Hrs_RN', 'Hrs_LPN', 'Hrs_CNA', 'Hrs_NAtrn']].copy()

In [63]:
staffing.head()

,WorkDate,PROVNUM,PROVNAME,STATE,MDScensus,Hrs_RN,Hrs_LPN,Hrs_CNA,Hrs_NAtrn
0,20180215,075001,ST JOSEPH'S CENTER,CT,247,99.18,268.12,597.19,0.0
1,20180117,075001,ST JOSEPH'S CENTER,CT,241,72.50,289.11,570.37,0.0
2,20180129,075001,ST JOSEPH'S CENTER,CT,241,81.26,286.49,592.83,0.0
3,20180121,075001,ST JOSEPH'S CENTER,CT,238,72.73,257.54,564.36,0.0
4,20180125,075001,ST JOSEPH'S CENTER,CT,242,91.01,285.34,599.67,0.0


In [64]:
staffing = staffing.apply(pd.to_numeric, errors='ignore')

In [65]:
copy = df[['facility_name', 'corp_type']].copy().drop_duplicates(subset='facility_name', keep='first')

In [66]:
copy = copy.reset_index()

In [67]:
staffing.head()

,WorkDate,PROVNUM,PROVNAME,STATE,MDScensus,Hrs_RN,Hrs_LPN,Hrs_CNA,Hrs_NAtrn
0,20180215,075001,ST JOSEPH'S CENTER,CT,247,99.18,268.12,597.19,0.0
1,20180117,075001,ST JOSEPH'S CENTER,CT,241,72.50,289.11,570.37,0.0
2,20180129,075001,ST JOSEPH'S CENTER,CT,241,81.26,286.49,592.83,0.0
3,20180121,075001,ST JOSEPH'S CENTER,CT,238,72.73,257.54,564.36,0.0
4,20180125,075001,ST JOSEPH'S CENTER,CT,242,91.01,285.34,599.67,0.0


In [68]:
staffing = copy.merge(staffing, left_on='facility_name', right_on='PROVNAME')

In [69]:
census_per_state = pd.DataFrame(staffing.groupby(by=['STATE', 'corp_type']).MDScensus.sum() * 4.8).reset_index()

In [71]:
complaints_per_capita = pd.DataFrame(df.groupby(by=['state', 'corp_type']).complaint.sum()).reset_index()

In [73]:
df2=complaints_per_capita.merge(census_per_state, left_index=True, right_index=True)

In [74]:
df2['complaints_per_capita'] = (df2['complaint'] / df2['MDScensus'])

In [75]:
df2['complaints_per_capita'] = df2['complaints_per_capita'] * 1000

In [78]:
df2 = df2.drop(['complaint', 'STATE', 'corp_type_y', 'MDScensus'], axis=1)

In [83]:
df2.to_csv('lollipopchart.csv')

## Deficiencies filed per capita

In [88]:
df3 = pd.DataFrame(df.groupby(by=['state', 'corp_type']).scope_severity.count()).reset_index()

In [90]:
df3 = df3.merge(census_per_state, left_index=True, right_index=True)

In [91]:
df3.head()

,state,corp_type_x,scope_severity,STATE,corp_type_y,MDScensus
0,AK,For profit,57,AK,For profit,248808.0
1,AK,Non profit/Government,641,AK,Non profit/Government,598310.4
2,AL,For profit,2682,AL,For profit,32600611.2
3,AL,Non profit/Government,483,AL,Non profit/Government,5222976.0
4,AR,For profit,4407,AR,For profit,19885156.8


In [92]:
df3['deficiencies_per_capita'] = (df3['scope_severity']/df3['MDScensus']) * 1000

In [95]:
df3 = df3[['state', 'corp_type_x', 'deficiencies_per_capita']]

In [96]:
df3=df3[df3.state != 'PR']

In [97]:
df3=df3[df3.state != 'DC']

In [102]:
df3

,state,corp_type_x,deficiencies_per_capita
0,AK,For profit,0.229092
1,AK,Non profit/Government,1.071350
2,AL,For profit,0.082268
3,AL,Non profit/Government,0.092476
4,AR,For profit,0.221623
5,AR,Non profit/Government,0.186361
6,AZ,For profit,0.127065
7,AZ,Non profit/Government,0.191343
8,CA,For profit,0.330670
9,CA,Non profit/Government,0.261556


In [98]:
df3.to_csv('multiples.csv')